<a href="https://colab.research.google.com/github/art-lmk/Email-Spam-Filter-app-Using-TfidfVectorizer/blob/main/Email_Spam_Filter_using_TF_IDF_Vectorization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
#import necessary dependencies and libraries
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
import warnings

warnings. filterwarnings('ignore')

In [7]:
#load the data

df= pd.read_csv('spam.csv')
df.head(10)



,Category,Message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
5,spam,FreeMsg Hey there darling it's been 3 week's n...
6,ham,Even my brother is not like to speak with me. ...
7,ham,As per your request 'Melle Melle (Oru Minnamin...
8,spam,WINNER!! As a valued network customer you have...
9,spam,Had your mobile 11 months or more? U R entitle...


In [8]:
#Exploratory Data Analysis(EDA)
df.isnull().sum()

df.drop_duplicates(inplace=True)


df['Category']= df['Category'].replace(['spam', 'ham'], ['Spam', 'Not Spam'])


#lets do label encoding
df.loc[df['Category']=='Spam','Category',]=0
df.loc[df['Category']=='Not Spam','Category',]=1
df.head(10)

,Category,Message
0,1,"Go until jurong point, crazy.. Available only ..."
1,1,Ok lar... Joking wif u oni...
2,0,Free entry in 2 a wkly comp to win FA Cup fina...
3,1,U dun say so early hor... U c already then say...
4,1,"Nah I don't think he goes to usf, he lives aro..."
5,0,FreeMsg Hey there darling it's been 3 week's n...
6,1,Even my brother is not like to speak with me. ...
7,1,As per your request 'Melle Melle (Oru Minnamin...
8,0,WINNER!! As a valued network customer you have...
9,0,Had your mobile 11 months or more? U R entitle...


In [9]:
#Feature Engineering

x= df['Message']
y= df['Category']

#print(y)
#print(x)


# Train-test split
x_train,x_test,y_train,y_test= train_test_split(x,y,test_size=0.2,random_state=3)

print(x_train.shape)
print(x_test.shape)

(4125,)
(1032,)


In [30]:
#convert now the data intonumerical values that can used as an input inside a model

f= TfidfVectorizer(min_df=1,stop_words='english',lowercase=True)
x_train_count= f.fit_transform(x_train)
x_test_count= f.transform(x_test)# we do not fit the test data


In [31]:
#convert y_train and y_test as integers(1 or 0)
y_train= y_train.astype('int')
y_test= y_test.astype('int')

print(x_train_count)#

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 31535 stored elements and shape (4125, 7431)>
  Coords	Values
  (0, 2400)	0.4689535663823655
  (0, 1247)	0.5538832733861689
  (0, 6605)	0.4898673616987752
  (0, 6692)	0.48303813512243965
  (1, 6492)	0.5755914257195885
  (1, 5859)	0.5964494866231046
  (1, 1592)	0.5594126567616489
  (2, 5786)	0.13178759818933938
  (2, 4038)	0.24488128414489752
  (2, 6925)	0.1186988586975311
  (2, 4089)	0.2196593047164816
  (2, 4522)	0.2363176943466334
  (2, 3695)	0.21568440262445418
  (2, 3252)	0.18176623831152225
  (2, 6828)	0.13848562282513652
  (2, 4084)	0.2644704778405301
  (2, 4519)	0.24488128414489752
  (2, 4520)	0.24488128414489752
  (2, 4393)	0.24488128414489752
  (2, 798)	0.24488128414489752
  (2, 2564)	0.19505111090227498
  (2, 25)	0.24488128414489752
  (2, 682)	0.22967525805125708
  (2, 7355)	0.21568440262445418
  (2, 5064)	0.22967525805125708
  :	:
  (4120, 2101)	0.3080717396234338
  (4120, 3921)	0.3922048767024034
  (4120, 3181)	0

The above output has integers as the message instead of a string.
The message is a score based on the vectorizer function(TFidfVectorizer).
Machines understand numbers

In [12]:
#train the data
model= LogisticRegression()
model.fit(x_train_count,y_train)

LogisticRegression()

In [13]:
#model evaluation on the trained data
predicted_data= model.predict(x_train_count)
accuracy= accuracy_score(y_train,predicted_data)
print(accuracy)

0.961939393939394


The accuracy score of the trained data is 96%

In [21]:
#model evaluation on the test data
predicted_data= model.predict(x_test_count)
accuracy= accuracy_score(y_test,predicted_data)
print(accuracy)


def predict(Message):
  input_message= cv.transform([Message]). toarray()
  result= model.predict(input_message)
  return result

0.9544573643410853


In [36]:
# Overwrite the Streamlit app code to a Python file
%%writefile app.py
import streamlit as st
import pickle
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
import joblib

# Load the trained model and vectorizer
try:
    model = joblib.load('model.pkl')
    f = joblib.load('vectorizer.pkl')
except FileNotFoundError:
    st.error("Model or vectorizer file not found. Please train and save them in the notebook first.")
    st.stop() # Stop the app if files are not found


def predict(Message):
  input_message_count = f.transform([Message])
  result = model.predict(input_message_count)[0]
  return result

st.title('Spam Detection')
message = st.text_area('ENTER THE MESSAGE HERE')

if st.button('PREDICT'):
  if message: # Check if the message is not empty
    result = predict(message)
    if result == 0:
      st.error('SPAM')
    else:
      st.success('NOT SPAM')
  else:
    st.warning('Please enter a message to predict.')

Overwriting app.py


In [26]:
!pip install pyngrok

In [37]:
from pyngrok import ngrok
from google.colab import userdata

# Get the authtoken from Colab secrets
NGROK_AUTH_TOKEN = userdata.get('NGROK_TOKEN')

# Authenticate ngrok
ngrok.set_auth_token(NGROK_AUTH_TOKEN)

# Connect to the Streamlit port
public_url = ngrok.connect(8501)
print(f"Streamlit app public URL: {public_url}")

Streamlit app public URL: NgrokTunnel: "https://a573e7f8d69c.ngrok-free.app" -> "http://localhost:8501"
